Author notes (Taegyoon Kim, taegyoon@psu.edu)


---


- This is a notebook for a BERT classifier for violent political rhetoric (https://osf.io/5ckw4/). 
- The input data is tweet text that contains one or more of the violent keywords extracted using the violent keyword extractor (https://github.com/taegyoon-kim/violent_political_rheotric_on_twitter/blob/master/violent_political_rhetoric_violent_keyword_extract.py). 
- The training data is available upon request via email. 
- The notebook will be fully available upon publication of the paper
- The classifier is trained on GPU provided Google Colaboratory



Drive mount

---







In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Packages


---



In [ ]:
# the classifier is bsed on simpletransformers package https://github.com/ThilinaRajapakse/simpletransformers

!pip install torch torchvision
!pip install transformers==2.10.0
!pip install seqeval
!pip install tensorboardx
!pip install simpletransformers==0.9.1

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np

import gc
import requests
import os

from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score
from scipy.special import softmax

import torch
print("Cuda available" if torch.cuda.is_available() is True else "CPU")
print("PyTorch version: ", torch.__version__)

Cuda available
PyTorch version:  1.9.0+cu102


Load data

---



In [ ]:
url= '' # training set

df = pd.read_csv(url)
df['text'] = df['status_final_text']
df['threat'] = df['final_binary'].astype(float)
df = df[['text','threat']]

df = df.sample(frac=1).reset_index(drop=True)

Performance metrics


---



In [ ]:
def report_results(A, B):
    A_name = A.name
    B_name = B.name
    
    df = pd.DataFrame({'A':A,
                       'B':B})
    df = df.dropna()
    A = df['A']
    B = df['B']
    
    acc = accuracy_score(B, A)
    f1 = f1_score(B, A)
    prec = precision_score(B, A)
    rec = recall_score(B, A)
    ROC = roc_auc_score(B, A)
    
    print('Candidate: '+A_name+' | Ground Truth: '+B_name+'\n')
    print('accuracy: %0.4f \nprecision: %0.4f \nrecall: %0.4f \nF1 score: %0.4f \nROC AUC: %0.4f \n' % (acc, prec, rec, f1, ROC))

    performance = [prec, rec, f1, ROC, acc]

    return performance

5-fold Cross-validation


---



In [ ]:
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',

   'fp16': False,
   'fp16_opt_level': 'O1',
   'max_seq_length': 250,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 2,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,

   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,

   'overwrite_output_dir': True,
   'reprocess_input_data': True   
}

from sklearn.model_selection import KFold
n = 5
kf = KFold(n_splits = n, random_state = 7, shuffle = True)

for train_index, val_index in kf.split(df):
  # splitting Dataframe (dataset not included)
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]
  # Defining Model
    model = ClassificationModel('bert', 'bert-base-uncased', args = args)
  # train the model
    model.train_model(train_df)
  # validate the model 
    #result, model_outputs, wrong_predictions = model.eval_model(val_df)
    #val_df['BERT_threat'] = np.argmax(model_outputs, axis = 1)
    predictions, raw_outputs = model.predict(val_df['text'])
    probabilities = softmax(raw_outputs, axis=1)
    val_df['BERT_threat_900'] = np.where(probabilities[:,1] >= 0.9, 1, 0)
    val_df['BERT_threat_925'] = np.where(probabilities[:,1] >= 0.925, 1, 0)
    val_df['BERT_threat_950'] = np.where(probabilities[:,1] >= 0.95, 1, 0)
  # performance
    performance_900 = report_results(val_df['BERT_threat_900'], val_df['threat'])
    performance_925 = report_results(val_df['BERT_threat_925'], val_df['threat'])
    performance_950 = report_results(val_df['BERT_threat_950'], val_df['threat'])
    print(performance_900)
    print(performance_925)
    print(performance_950)